# OpenAI Agents SDK 101: Build Tools‑Enabled AI Agents (Python)

Learn how to:
- Securely load your OpenAI API key from AWS Secrets Manager
- Initialize the OpenAI Python SDK
- Build an agent that can use tools (function calling)
- Maintain conversation state (memory) across turns
- Optionally explore the Assistants API and retrieval basics

This notebook is beginner‑friendly and focused on practical, copy‑pasteable examples.

## Prerequisites
- Python 3.9+ recommended
- An AWS account with a secret named `openai` in AWS Secrets Manager. The secret value should be JSON like:
  ```json
  {
    "api_key": "sk-..."
  }
  ```
- AWS credentials configured locally (e.g., via `aws configure` or environment variables) with permission to read Secrets Manager.
- Python packages: `openai` (>=1.0), `boto3`.

If you need to install packages in this environment, run:

In [ ]:
# If needed, uncomment and run:
# %pip install --upgrade openai boto3
# For Assistants API file features you may also want: 
# %pip install python-dotenv tiktoken


## Load API Key from AWS Secrets Manager
We will use AWS Secrets Manager to retrieve the OpenAI API key. This helper uses the `openai` secret and expects an `api_key` field.

In [ ]:
import json
import boto3
from botocore.exceptions import ClientError

def get_secret(secret_name: str, region_name: str = "us-east-1") -> dict:
    """Fetch and return a JSON secret from AWS Secrets Manager.
    Expects the secret's SecretString to be JSON.
    """
    session = boto3.session.Session()
    client = session.client(service_name="secretsmanager", region_name=region_name)
    try:
        resp = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        raise RuntimeError(f"Failed to retrieve secret '{secret_name}': {e}")
    secret_str = resp.get("SecretString")
    if not secret_str:
        raise RuntimeError("SecretString is empty or missing.")
    return json.loads(secret_str)

# Retrieve the OpenAI API key from Secret 'openai'
openai_secret = get_secret('openai')
openai_api_key = openai_secret['api_key']
print('Loaded OpenAI API key from AWS Secrets Manager (masked).')


## Initialize the OpenAI SDK
We'll use the Python SDK (v1+) with the `OpenAI` client.

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)
# Pick a reasonably priced, tool-capable model. Adjust as needed.
DEFAULT_MODEL = 'gpt-4o-mini'
print('OpenAI client initialized.')


## Hello, Agent (Basic Chat)
A minimal call using the Chat Completions API.

In [ ]:
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[
        {"role": "system", "content": "You are a concise, helpful teaching assistant."},
        {"role": "user", "content": "Explain what an AI agent is in one sentence."},
    ],
)
print(response.choices[0].message.content)


## Tool Use (Function Calling)
Agents become powerful when they can use tools: call your functions with structured inputs.
Below we define two example tools and let the model decide when to use them.

In [ ]:
import math
from datetime import datetime

def calculator(operation: str, a: float, b: float) -> float:
    if operation == 'add':
        return a + b
    if operation == 'sub':
        return a - b
    if operation == 'mul':
        return a * b
    if operation == 'div':
        return a / b
    if operation == 'pow':
        return math.pow(a, b)
    raise ValueError('Unsupported operation')

def get_time(tz: str = 'UTC') -> str:
    # Simple placeholder: ignores tz and returns ISO time.
    return datetime.utcnow().isoformat() + 'Z'

tools = [
    {
        'type': 'function',
        'function': {
            'name': 'calculator',
            'description': 'Perform basic arithmetic operations.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'operation': { 'type': 'string', 'enum': ['add','sub','mul','div','pow'] },
                    'a': { 'type': 'number' },
                    'b': { 'type': 'number' }
                },
                'required': ['operation','a','b']
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'get_time',
            'description': 'Get current time in ISO8601 (UTC).',
            'parameters': {
                'type': 'object',
                'properties': { 'tz': { 'type': 'string' } },
                'required': []
            }
        }
    }
]

messages = [
    { 'role': 'system', 'content': 'You are a helpful agent. Use tools when helpful.' },
    { 'role': 'user', 'content': 'What is (3^4 + 10) / 2? Also, what time is it now?' },
]

resp = client.chat.completions.create(model=DEFAULT_MODEL, messages=messages, tools=tools, tool_choice='auto')
msg = resp.choices[0].message
print('Model draft:
', msg)

# If the model requested tool calls, execute them and feed results back.
if msg.tool_calls:
    # IMPORTANT: append the assistant message WITH tool_calls before any tool responses.
    messages.append({
        'role': 'assistant',
        'content': msg.content or '',
        'tool_calls': [
            {
                'id': tc.id,
                'type': 'function',
                'function': { 'name': tc.function.name, 'arguments': tc.function.arguments or '{}' }
            } for tc in (msg.tool_calls or [])
        ]
    })
    for call in msg.tool_calls:
        name = call.function.name
        args = json.loads(call.function.arguments or '{}')
        if name == 'calculator':
            result = calculator(**args)
        elif name == 'get_time':
            result = get_time(**args)
        else:
            result = f"Unknown tool: {name}"
        messages.append({ 'role': 'tool', 'tool_call_id': call.id, 'content': json.dumps({'result': result}) })
    # Ask the model to produce a final answer using tool outputs
    final = client.chat.completions.create(model=DEFAULT_MODEL, messages=messages)
    print('
Final answer:
', final.choices[0].message.content)
else:
    print('
No tool calls. Assistant says:
', msg.content)


## Minimal Reusable Agent Wrapper
A small helper that preserves message history and tools across turns.

In [ ]:
class Agent:
    def __init__(self, client: OpenAI, model: str, system_prompt: str = 'You are a helpful agent.', tools: list | None = None):
        self.client = client
        self.model = model
        self.messages = [ { 'role': 'system', 'content': system_prompt } ]
        self.tools = tools or []

    def ask(self, content: str) -> str:
        self.messages.append({ 'role': 'user', 'content': content })
        resp = self.client.chat.completions.create(model=self.model, messages=self.messages, tools=self.tools or None, tool_choice='auto' if self.tools else 'none')
        msg = resp.choices[0].message
        self.messages.append({ 'role': 'assistant', 'content': msg.content or '' , 'tool_calls': msg.tool_calls })
        if msg.tool_calls:
            for call in msg.tool_calls:
                name = call.function.name
                args = json.loads(call.function.arguments or '{}')
                if name == 'calculator':
                    result = calculator(**args)
                elif name == 'get_time':
                    result = get_time(**args)
                else:
                    result = f"Unknown tool: {name}"
                self.messages.append({ 'role': 'tool', 'tool_call_id': call.id, 'content': json.dumps({'result': result}) })
            followup = self.client.chat.completions.create(model=self.model, messages=self.messages)
            final_text = followup.choices[0].message.content or ''
            self.messages.append({ 'role': 'assistant', 'content': final_text })
            return final_text
        return msg.content or ''

agent = Agent(client, DEFAULT_MODEL, tools=tools)
print(agent.ask('Compute (5*7)+11 and tell me current time.'))
print(agent.ask('Great, remind me what we just calculated.'))


## Optional: Assistants API Preview
The Assistants API provides hosted tools like code interpreter and retrieval.
Below is a minimal sketch; ensure your `openai` package is recent.

In [ ]:
# Note: This requires an SDK version that includes beta.assistants.*
# from openai import OpenAI
# client = OpenAI(api_key=openai_api_key)
#
# assistant = client.beta.assistants.create(
#     name="Math + Time Assistant",
#     instructions="You can compute and tell time. Be concise.",
#     model=DEFAULT_MODEL,
#     tools=[{ 'type': 'code_interpreter' }]
# )
#
# thread = client.beta.threads.create()
# client.beta.threads.messages.create(thread_id=thread.id, role="user", content="What's 123*45?")
# run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
# # Poll run status until completed, then read messages:
# import time as _t
# while True:
#     r = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
#     if r.status in ['completed','failed','cancelled','expired']:
#         break
#     _t.sleep(1)
# messages = client.beta.threads.messages.list(thread.id)
# print(messages.data[0].content[0].text.value)


## Optional: Retrieval (RAG) Basics with Assistants
High level outline (requires file upload permissions):
- Upload files with `client.files.create(...)`
- Attach files to an assistant or a vector store
- Ask questions; the assistant retrieves relevant chunks

See OpenAI docs for details and quotas.

## Best Practices
- Keep system instructions short and specific
- Constrain tools with strict JSON Schemas
- Log tool I/O and handle errors defensively
- Cache model outputs where possible to control cost
- Prefer smaller, cheaper models for tool routing; escalate only when needed
- Never print secrets in logs or notebooks

You now have a working template to build agents that call your Python functions, maintain memory, and optionally leverage the Assistants API.